## The `Bool` Widgets

+ CheckBox
+ ToggleButton

These widgets can be used to represent a Boolean value. The idea is pretty simple, the widget can be in one of two states which represent the two boolean values.

    Checked / On : True
    Unchecked / Off : False

In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE FlexibleContexts #-}
import IHaskell.Display.Widgets
import Data.Text (pack, unpack)
import Text.Printf (printf)

### Simple demonstration
We are going to create one of each widget.

The valid widget doesn't receive any input, but it displays the status of a boolean variable.

In [2]:
-- Check box
chk <- mkCheckBox

-- Toggle button
tgb <- mkToggleButton

-- Valid widget: Displaying booleans conveniently
vld <- mkValid

Below, we represent one boolean using a checkbox, and the other using a toggle button. The valid widget displays the result of applying the logical and (`&&`) between the two.

In [3]:
-- Display the widgets
setField @Description tgb "Button"
chk
tgb
vld

To make the `Valid` widget display the result of the computation, we have to get the value of the Checkbox and the value of the ToggleButton, and then set the value of the Valid widget to the result.

In [4]:
b1 <- getField @BoolValue chk
b2 <- getField @BoolValue tgb
setField @BoolValue vld (b1 && b2)

You can change the value of the widgets clicking in them, and then run the cell above again. If both widgets are checked, the valid widget should appear with a green check.

But having to run the cell every time we click on the widgets is a bit cumbersome... Why not making it change auto*magic*ally? To do this, we will use a **handler**. A *handler* is a method that is called every time something happens. E.g: every time the button is clicked, or every time its associated value is changed on the frontend. These handlers are functions of type `IO ()`.

Now, we are going to create a handler that is called every time one of the two widget changes and updates the `Valid` widget. We will also set a useful description.

In [5]:
setField @Description chk "b1"
setField @Description tgb "b2"
setField @Description vld "b1 && b2:"

ourBoolHandler :: IO ()
ourBoolHandler = do
  b1 <- getField @BoolValue chk
  b2 <- getField @BoolValue tgb
  setField @BoolValue vld (b1 && b2)

setField @ChangeHandler chk ourBoolHandler
setField @ChangeHandler tgb ourBoolHandler

### Extended example

Let's try to create a graphical 8-bit-binary to decimal converter. We'll represent seven bits using `ToggleButton` widgets, and the negative bit using a `CheckBox`. The binary number is represented using 1+7-bit sign-and-magnitude representation for simplicity.

Boxes are used to layout the widgets in an appealing manner, and the output widget is used to display the result.

In [6]:
-- First, some library functions
import Control.Monad (replicateM, forM_)
import Data.IORef
import IHaskell.Display (plain)
import qualified IHaskell.Display.Widgets.Layout as L

Now, we create a `CheckBox` and seven `ToggleButton`s.

In [8]:
sign <- mkCheckBox
bits <- replicateM 7 mkToggleButton

setField @Description sign "Negative"
forM_ bits $ \t -> do
  setField @ButtonStyleField t PrimaryButton
  -- setField @BorderRadius t 20

Then we create a `FlexBox` to hold the widgets, and an `HTMLWidget` to display the output.

In [11]:
box <- mkVBox
out <- mkHTML

-- Sub-containers
box1 <- mkBox
setField @Children box1 [ChildWidget sign, ChildWidget out]
box2 <- mkBox
setField @Children box2 (map ChildWidget $ reverse bits)

-- Add widgets to the container
setField @Children box (map ChildWidget [box1, box2])

-- Add some UI chrome
setField @BoxStyle box InfoBox
layout <- getField @Layout out
setField @L.Width layout $ Just "100px"
setField @L.Height layout $ Just "30px"
setField @L.Margin layout $ Just "10px"
setField @L.BorderTop layout $ Just "4px groove"

-- Display the container
box

Now, we implement the logic of our converter, and make it send the output to the `HTMLWidget` we created above.

In [12]:
import Control.Arrow (first, second)

-- Mutable value, with a sign bit
val <- newIORef (0 :: Int, False)

-- Helper function to redraw output
refresh :: (Int, Bool) -> IO ()
refresh (x, b) = 
  let val = x * if b then (-1) else 1
      fmt = "<div align=\"center\"><b>%d</b></div>"
  in setField @StringValue out (pack $ printf fmt val)

setField @ChangeHandler sign $ do
  -- Change sign for value
  modifyIORef val (second not)
  -- Redraw output
  readIORef val >>= refresh
  
setField @StringValue out "<div align=\"center\"><b>%d</b></div>"

forM_ (zip bits (iterate (*2) 1)) $ \(t, n) -> do
  setField @Description t "0"
  setField @ChangeHandler t $ do
    f <- getField @BoolValue t
    setField @Description t (if f then "1" else "0")
    modifyIORef val (first $ if f then (+n) else (\x->x-n))
    readIORef val >>= refresh